In [1]:
pip install requests beautifulsoup4


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re  # Digunakan untuk mencari ID yang diawali "article-"

def scrape_jptiik(max_articles=5, output_file="jptiik_articles.csv"):
    url = 'https://j-ptiik.ub.ac.id/index.php/j-ptiik/'

    # Konfigurasi Selenium
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Jalankan tanpa membuka browser
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Hindari deteksi bot
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(url)

    # Tunggu agar JavaScript dapat memuat halaman sepenuhnya
    driver.implicitly_wait(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # Cari semua artikel yang ada di daftar
    articles = soup.find_all('div', class_='obj_article_summary')

    # Simpan hasil ke dalam list
    scraped_data = []

    count = 0  # Untuk menghitung berapa artikel yang sudah diambil

    for article in articles:
        if count >= max_articles:
            break  # Berhenti jika sudah mencapai jumlah yang diinginkan

        # Ambil judul artikel (dari <a> yang memiliki id diawali "article-")
        title_tag = article.find('a', id=re.compile(r'^article-\d+'))  # Cari <a> dengan id "article-XXXX"
        title = title_tag.text.strip() if title_tag else 'Tidak ditemukan'

        # Ambil nama penulis
        author_tag = article.find('div', class_='authors')
        author = author_tag.text.strip() if author_tag else 'Tidak ditemukan'

        scraped_data.append([title, author])  # Simpan ke dalam list

        count += 1  # Tambah counter setelah mengambil satu artikel

    # Simpan ke file CSV
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Judul", "Penulis"])  # Header CSV
        writer.writerows(scraped_data)  # Data

    print(f"Scraping selesai! Data disimpan ke {output_file}")

if __name__ == '__main__':
    # Masukkan jumlah artikel yang ingin diambil
    scrape_jptiik(max_articles=2000, output_file="jptiik_articles.csv")



Scraping selesai! Data disimpan ke jptiik_articles.csv


In [4]:


import requests
import csv
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Konfigurasi header agar tidak terdeteksi sebagai bot
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://j-ptiik.ub.ac.id/index.php/j-ptiik/issue/archive",
    "Connection": "keep-alive"
}

# Masukkan cookies dari browser jika perlu (dapatkan dari browser dengan EditThisCookie)
COOKIES = {
    "PHPSESSID": "your_session_id_here",
    "other_cookie": "value_here"
}

# Gunakan proxy jika diperlukan
PROXIES = {
    "http": "http://your_proxy_here",
    "https": "https://your_proxy_here"
}

BASE_URL = "https://j-ptiik.ub.ac.id"
ARCHIVE_URL = "https://j-ptiik.ub.ac.id/index.php/j-ptiik/issue/archive"
OUTPUT_CSV = "jurnal_ktiik.csv"

# Fungsi untuk mengambil halaman menggunakan requests (fallback ke Selenium jika gagal)
def get_html(url, use_selenium=False):
    """Mengambil HTML dari halaman, dengan opsi menggunakan Selenium jika requests gagal."""
    try:
        response = requests.get(url, headers=HEADERS, cookies=COOKIES, proxies=PROXIES, timeout=10)
        if response.status_code == 200:
            return response.text
        elif response.status_code == 403:
            print(f"⚠️ 403 Forbidden: Menggunakan Selenium untuk {url}")
            return get_html_selenium(url)
        else:
            print(f"❌ Gagal mengambil halaman {url}: {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"❌ Request error: {e}")
        return None

# Fungsi untuk mengambil HTML menggunakan Selenium
def get_html_selenium(url):
    """Menggunakan Selenium untuk mendapatkan HTML dari halaman yang diproteksi."""
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Tampilkan browser
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    driver.get(url)
    time.sleep(5)  # Tunggu agar halaman benar-benar termuat
    
    html = driver.page_source
    driver.quit()
    return html

# Fungsi mengambil link edisi jurnal (bulan penerbitan)
def get_monthly_links():
    """Mengambil semua link bulan penerbitan dari halaman arsip."""
    html = get_html(ARCHIVE_URL)
    if not html:
        return []

    soup = BeautifulSoup(html, 'html.parser')
    monthly_links = [(month.text.strip(), BASE_URL + month['href']) for month in soup.find_all('a', class_='title')]
    return monthly_links

# Fungsi mengambil link artikel dalam satu edisi jurnal
def get_article_links(issue_url):
    """Mengambil semua link artikel dari halaman edisi jurnal."""
    html = get_html(issue_url)
    if not html:
        return []

    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find_all('a', id=re.compile(r'^article-\d+'))
    article_links = [(article.text.strip(), BASE_URL + article['href']) for article in articles]
    
    return article_links

# Fungsi mengambil detail artikel (judul & author)
def scrape_article_details(article_url):
    """Scraping detail artikel (judul dan penulis) menggunakan Selenium jika perlu."""
    html = get_html(article_url, use_selenium=True)
    if not html:
        return "Tidak ditemukan", "Tidak ditemukan"

    soup = BeautifulSoup(html, 'html.parser')

    title_tag = soup.find('h1', class_='page_title')  # Coba cari elemen judul
    title = title_tag.text.strip() if title_tag else "Tidak ditemukan"

    author_tag = soup.find('div', class_='authors')  # Cari elemen author
    authors = author_tag.text.strip() if author_tag else "Tidak ditemukan"

    return title, authors

# Fungsi utama scraping semua artikel
def scrape_articles():
    """Scraping semua artikel dari semua bulan penerbitan dan simpan dalam satu CSV."""
    monthly_links = get_monthly_links()
    all_data = []

    for month_name, month_url in monthly_links:
        print(f"🔍 Scraping bulan: {month_name}")
        article_links = get_article_links(month_url)

        for article_title, article_url in article_links:
            title, authors = scrape_article_details(article_url)
            all_data.append([month_name, title, authors])
            print(f"✅ {title} - {authors}")
            time.sleep(1)  # Hindari pemblokiran dengan delay

    save_to_csv(all_data)

# Fungsi menyimpan hasil scraping ke CSV
def save_to_csv(data):
    """Simpan hasil scraping ke dalam satu file CSV."""
    with open(OUTPUT_CSV, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Bulan", "Judul", "Penulis"])
        writer.writerows(data)

    print(f"✅ Semua data disimpan di {OUTPUT_CSV}")

# Eksekusi kode utama
if __name__ == '__main__':
    scrape_articles()
    print("✅ Scraping selesai!")


❌ Request error: HTTPSConnectionPool(host='j-ptiik.ub.ac.id', port=443): Max retries exceeded with url: /index.php/j-ptiik/issue/archive (Caused by ProxyError('Unable to connect to proxy', NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001ED7680E390>: Failed to resolve 'your_proxy_here' ([Errno 11001] getaddrinfo failed)")))
✅ Semua data disimpan di jurnal_ktiik.csv
✅ Scraping selesai!
